## 1. Read rawdata => order

- order.csv format:

| # |column_name | type | value |
| ---- | ---- | ---- | ---- |
| 0| |  |  |
| 1| id | int | ... |
| 2| created_at | datetime | order created_at |
| 3| closed_at | datetime | order closed_at |
| 4| processed_at | datetime | order processed_at |
| 5| total_price | USD | order total_price |
| 6| financial_status| enum | paid/refunded/partially_refunded |
| 7| fulfillment_status| enum | fulfilled |
| 8| line_items| json |  |
| 9| fulfillments| json | |
|10| refunds| json | |
|11| discount_applications| json | |
|12| customer| json | |

- line_items:

- fulfillments:
- refunds:
- discount_applications:
- customer:
| # |column_name | type | value |
| ---- | ---- | ---- | ---- |
| 0| |  | {"id":150444605461,"created_at":"2018-01-02T12:32:42-08:00","country":"United States"} |
| 1| id| int | ... |
| 2| created_at| datetime | user created_at |
| 3| country| string | user's country |



In [1]:
# !pip install matplotlib

In [2]:
import pandas as pd
import numpy as np
from datetime import datetime, date
import matplotlib.pyplot as plt
# import seaborn as sns

In [3]:
order_df = pd.read_csv('../data-raw/orders.csv', parse_dates=[1,11], infer_datetime_format=True)

In [4]:
import json

def dataframe_flatten(df, column):
    if column not in df.columns:
        return df
    if isinstance(df[column][0], str):
        data_fillna = df[column].fillna('{}')
        data_json = data_fillna.apply(json.loads)
    else:
        data_fillna = df[column].fillna({})
        data_json = data_fillna
    new_df = pd.json_normalize(data_json)
    new_df = new_df.add_prefix('{0}.'.format(column))
    # print(new_df.head())
    return pd.concat([df.drop(column, 1), new_df], axis=1)

## Flatten order data => customer

In [5]:
# order_df['customer'] is json -> dict format => can parse and add to order_df directly 
# order_df =  dataframe_flatten(order_df, 'line_items')
# order_df =  dataframe_flatten(order_df, 'fulfillments')
# order_df =  dataframe_flatten(order_df, 'refunds')
# order_df =  dataframe_flatten(order_df, 'discount_applications')
order_df =  dataframe_flatten(order_df, 'customer')

In [6]:
# Deprecated
# order_df['customer'].apply(pd.Series)[0].apply(pd.Series)
# def parse_list_json_to_dataframe(data):
    

# order_df['customer'].fillna('{}')
# order_df['customer'].fillna('{"id":-1}').apply(pd.DataFrame.from_records)
# order_df['customer'].fillna('{"id":-1}').apply(pd.json_normalize)

# Target_df = pd.json_normalize(order_df['customer'])
# order_df['customer'].isna().sum()
# print (order_df['customer'].fillna('{"id":-1,"created_at":"2018-01-01T17:23:26-08:00","country":"United States"}').isna().sum())
# print (order_df['customer'][0])
# pd.json_normalize(order_df['customer'].fillna('{"id":-1,"created_at":"2018-01-01T17:23:26-08:00","country":"United States"}'))

# order_df['customer'] = order_df['customer'].fillna('{}')
# order_df['customer'] = order_df['customer'].apply(json.loads)
# pd.json_normalize(order_df['customer']).add_prefix('customer.')

# pd.concat([order_df.drop(['customer'], axis=1), order_df['customer'].apply(pd.Series)], axis=1)
order_df.head()

,id,created_at,closed_at,processed_at,total_price,financial_status,fulfillment_status,line_items,fulfillments,refunds,discount_applications,customer.id,customer.created_at,customer.country
0,20570,2018-01-01 17:24:47+00:00,2018-01-02T18:02:30.000Z,2018-01-01T17:24:47.000Z,449.97,paid,fulfilled,"[{""title"":""Vertigo™ Semi-Hollow"",""quantity"":1,...","[{""created_at"":""2018-01-02T18:02:30-08:00"",""li...",[],[],1.494266e+11,2018-01-01T17:23:26-08:00,United States
1,20556,2018-01-02 10:56:19+00:00,2018-01-02T18:02:32.000Z,2018-01-02T10:56:19.000Z,249.99,paid,fulfilled,"[{""title"":""Vertigo™ Semi-Hollow"",""quantity"":1,...","[{""created_at"":""2018-01-02T18:02:32-08:00"",""li...",[],[],1.503216e+11,2018-01-02T10:55:03-08:00,United States
2,20554,2018-01-02 11:45:26+00:00,2018-01-02T18:02:33.000Z,2018-01-02T11:45:26.000Z,NaN,paid,fulfilled,"[{""title"":""The FlyBy"",""quantity"":1,""sku"":""EFX-...","[{""created_at"":""2018-01-02T18:02:33-08:00"",""li...",[],"[{""type"":""manual"",""value"":""229.99"",""value_type...",1.503838e+11,2018-01-02T11:45:05-08:00,United States
3,20573,2018-01-01 12:38:56+00:00,2018-01-02T18:02:34.000Z,2018-01-01T12:38:56.000Z,249.99,paid,fulfilled,"[{""title"":""Vertigo™ Electric Guitar"",""quantity...","[{""created_at"":""2018-01-02T18:02:34-08:00"",""li...",[],[],1.490918e+11,2018-01-01T12:34:09-08:00,United States
4,20575,2017-12-31 16:42:06+00:00,2018-01-02T18:02:36.000Z,2017-12-31T16:42:06.000Z,172.67,paid,fulfilled,"[{""title"":""The FlyBy"",""quantity"":1,""sku"":""EFX-...","[{""created_at"":""2018-01-02T18:02:36-08:00"",""li...",[],"[{""type"":""discount_code"",""value"":""30.0"",""value...",6.553114e+09,2017-08-02T12:12:35-07:00,United States


## Flatten order data => line_items + fulfillments

In [7]:
# as consider checking if line_items is contain inside fulfillments
import json
fulfillment_status = order_df['fulfillment_status'].isnull()
# print (fulfillment_status, len(fulfillments))
line_items = order_df['line_items']
fulfillments = order_df['fulfillments']
first_i = -1
for i in range(len(fulfillments)): 
    try:
        if i in [3943, 3692]:
            continue
        if fulfillment_status[i]:
            continue
        if order_df['fulfillment_status'][i] == 'partial':
            continue
        fulfillitems = sum(list(map(lambda x: x.get("line_items", []), json.loads(fulfillments[i]))), [])
        if sorted(json.loads(line_items[i]), key=lambda x: x['title']) != sorted(fulfillitems, key=lambda x: x['title']):
            first_i = i
            break
    except Exception as ex:
        print(ex, i, fulfillment_status[i], fulfillments[i], line_items[i])

print(first_i)
if first_i >= 0:
    # first_i = next((i for i in range(len(fulfillments)) if not fulfillment_status[i] and len(json.loads(fulfillments[i])) == 0 or sorted(json.loads(line_items[i]), key=lambda x: x['title']) != sorted(json.loads(fulfillments[i])[0].get("line_items", []), key=lambda x: x['title'])), None)
    print(fulfillments[first_i], fulfillment_status[first_i],  order_df['fulfillment_status'][first_i])
    print(line_items[first_i])
    print(line_items[first_i] in fulfillments[first_i])

    print(list(map(lambda x: x.get("line_items", []), json.loads(fulfillments[first_i]))))

    fulfillitems = sorted(sum(list(map(lambda x: x.get("line_items", []), json.loads(fulfillments[first_i]))), []), key=lambda x: x['title'])
    items = sorted(json.loads(line_items[first_i]), key=lambda x: x['title'])
    print(items, '\n', fulfillitems)

    print (items == fulfillitems )

    test_1 = json.dumps(items).split(',')
    test_2 = json.dumps(fulfillitems).split(',')
    for _ in range(min(len(test_1), len(test_2))):
        if test_1[_] == test_2[_]:
            continue
        print (test_1[_], test_2[_])
    # print(sorted(json.loads(line_items[first_i]), key=lambda x: x['title']) == sorted(json.loads(fulfillments[first_i])[0]["line_items"], key=lambda x: x['title']))
    # print(json.dumps(json.loads(line_items[first_i]), sort_keys=True), '\n', json.dumps(json.loads(fulfillments[first_i])[0]["line_items"], sort_keys=True))

    # order_df.head()

-1


As the result, can see that the data in line_items and fulfillments is same => drop fulfillments

In [8]:
# pd.concat([order_df.drop(['customer'], axis=1), order_df['customer'].apply(pd.Series)], axis=1)
if 'fulfillments' in order_df:
    order_df = order_df.drop('fulfillments', axis=1)
# https://stackoverflow.com/questions/32468402/how-to-explode-a-list-inside-a-dataframe-cell-into-separate-rows
    
# https://towardsdatascience.com/how-to-quickly-create-and-unpack-lists-with-pandas-d0e78e487c75
if isinstance(order_df['line_items'][0], str):
#     order_df.reset_index(inplace=True, drop=True)
    order_df['line_items'] = order_df['line_items'].apply(json.loads)
    order_df = order_df.explode('line_items')
#     order_df.reset_index(inplace=True, drop=True)


In [9]:
order_df.reset_index(inplace=True, drop=True)
order_df =  dataframe_flatten(order_df, 'line_items')
order_df

,id,created_at,closed_at,processed_at,total_price,financial_status,fulfillment_status,refunds,discount_applications,customer.id,customer.created_at,customer.country,line_items.title,line_items.quantity,line_items.sku,line_items.grams,line_items.price,line_items.total_discount,line_items.fulfillment_status,line_items.discount_allocations
0,20570,2018-01-01 17:24:47+00:00,2018-01-02T18:02:30.000Z,2018-01-01T17:24:47.000Z,449.97,paid,fulfilled,[],[],1.494266e+11,2018-01-01T17:23:26-08:00,United States,Vertigo™ Semi-Hollow,1,M80-VHB-BLK,20412,249.99,0.00,fulfilled,[]
1,20570,2018-01-01 17:24:47+00:00,2018-01-02T18:02:30.000Z,2018-01-01T17:24:47.000Z,449.97,paid,fulfilled,[],[],1.494266e+11,2018-01-01T17:23:26-08:00,United States,The Tick™ 2.0 Black,1,M80-TICK-V2-BLK,3175,89.99,0.00,fulfilled,[]
2,20570,2018-01-01 17:24:47+00:00,2018-01-02T18:02:30.000Z,2018-01-01T17:24:47.000Z,449.97,paid,fulfilled,[],[],1.494266e+11,2018-01-01T17:23:26-08:00,United States,Pedalboard Lite (Silver),1,PFX-PB-LT-SLV,912,109.99,0.00,fulfilled,[]
3,20556,2018-01-02 10:56:19+00:00,2018-01-02T18:02:32.000Z,2018-01-02T10:56:19.000Z,249.99,paid,fulfilled,[],[],1.503216e+11,2018-01-02T10:55:03-08:00,United States,Vertigo™ Semi-Hollow,1,M80-VHB-BLK,20412,249.99,0.00,fulfilled,[]
4,20554,2018-01-02 11:45:26+00:00,2018-01-02T18:02:33.000Z,2018-01-02T11:45:26.000Z,NaN,paid,fulfilled,[],"[{""type"":""manual"",""value"":""229.99"",""value_type...",1.503838e+11,2018-01-02T11:45:05-08:00,United States,The FlyBy,1,EFX-FLY-BLK,7257,229.99,229.99,fulfilled,"[{'amount': '229.99', 'discount_application_in..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7689,400417,2021-04-02 06:03:06+00:00,2021-04-02T17:02:31.000Z,2021-04-02T06:03:05.000Z,269.99,paid,fulfilled,[],[],1.122758e+12,2019-02-04T14:20:00-08:00,United States,"Pedalboard Large, Black and Pro Accessory Case...",1,PFX-PB-L-BLK-BDL,0,269.99,0.00,fulfilled,[]
7690,400436,2021-04-02 07:16:47+00:00,2021-04-02T17:02:33.000Z,2021-04-02T07:16:46.000Z,219.48,paid,fulfilled,[],[],6.553521e+09,2017-08-02T12:42:03-07:00,United States,"Classic Tick Accessory Case 2.0, Black",1,M80-TICK-V2-BLK,3175,89.99,0.00,fulfilled,[]
7691,400436,2021-04-02 07:16:47+00:00,2021-04-02T17:02:33.000Z,2021-04-02T07:16:46.000Z,219.48,paid,fulfilled,[],[],6.553521e+09,2017-08-02T12:42:03-07:00,United States,"Pedalboard Lite, Black",1,PFX-PB-LT-BLK,912,109.99,0.00,fulfilled,[]
7692,400451,2021-04-02 09:39:12+00:00,2021-04-02T17:02:34.000Z,2021-04-02T09:39:11.000Z,329.99,paid,fulfilled,[],[],5.074619e+12,2021-04-02T10:37:55-07:00,United States,"Classic Dual Electric Guitar Case, Black",1,M80-2G-BLK,20412,329.99,0.00,fulfilled,[]


In [10]:
# order_df['discount_applications'] = order_df['discount_applications'].apply(lambda x: len(json.loads(x)) > 0)
order_df.head()

,id,created_at,closed_at,processed_at,total_price,financial_status,fulfillment_status,refunds,discount_applications,customer.id,customer.created_at,customer.country,line_items.title,line_items.quantity,line_items.sku,line_items.grams,line_items.price,line_items.total_discount,line_items.fulfillment_status,line_items.discount_allocations
0,20570,2018-01-01 17:24:47+00:00,2018-01-02T18:02:30.000Z,2018-01-01T17:24:47.000Z,449.97,paid,fulfilled,[],[],1.494266e+11,2018-01-01T17:23:26-08:00,United States,Vertigo™ Semi-Hollow,1,M80-VHB-BLK,20412,249.99,0.00,fulfilled,[]
1,20570,2018-01-01 17:24:47+00:00,2018-01-02T18:02:30.000Z,2018-01-01T17:24:47.000Z,449.97,paid,fulfilled,[],[],1.494266e+11,2018-01-01T17:23:26-08:00,United States,The Tick™ 2.0 Black,1,M80-TICK-V2-BLK,3175,89.99,0.00,fulfilled,[]
2,20570,2018-01-01 17:24:47+00:00,2018-01-02T18:02:30.000Z,2018-01-01T17:24:47.000Z,449.97,paid,fulfilled,[],[],1.494266e+11,2018-01-01T17:23:26-08:00,United States,Pedalboard Lite (Silver),1,PFX-PB-LT-SLV,912,109.99,0.00,fulfilled,[]
3,20556,2018-01-02 10:56:19+00:00,2018-01-02T18:02:32.000Z,2018-01-02T10:56:19.000Z,249.99,paid,fulfilled,[],[],1.503216e+11,2018-01-02T10:55:03-08:00,United States,Vertigo™ Semi-Hollow,1,M80-VHB-BLK,20412,249.99,0.00,fulfilled,[]
4,20554,2018-01-02 11:45:26+00:00,2018-01-02T18:02:33.000Z,2018-01-02T11:45:26.000Z,NaN,paid,fulfilled,[],"[{""type"":""manual"",""value"":""229.99"",""value_type...",1.503838e+11,2018-01-02T11:45:05-08:00,United States,The FlyBy,1,EFX-FLY-BLK,7257,229.99,229.99,fulfilled,"[{'amount': '229.99', 'discount_application_in..."


## Export data to csv

In [11]:
# order_pd.to_csv('../data/order_after_preprocess.csv', encoding='utf-8', index=False)
order_df.to_csv('../data-processed/orders_p1.csv', encoding='utf-8', index=False)
